### GridSearchCV- 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에

하이퍼 파라미터는 머신러닝 알고리즘을 구성하는 주요 구성 요소이며,이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있습니다.</br>
사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor와 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출 할 수 있는 방안을 제공합니다.</br>
(촘촘하게 파라미터를 입력하면서 테스트를 하는 방식입니다.)

<GridSearchCV 클래스의 생성자로 들어가는 주요 파라미터></br>
* estimator: clssifier, regressor, pipeline이 사용될 수 있습니다.
* param_grid: key + 리스트 값을 가지는 딕셔너리가 주어집니다. estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정합니다.
* scoring: 예측 성능을 측정할 평가 방법을 지정합니다. 보통은 사이킷런의 성능 평가 지표를 지정하는 문자열로 지정하거나 별도의 성능 평가 지표 함수로 지정할 수 있습니다.
* cv: 교차 검증을 위해 분할되는 학습/데이터 세트의 개수를 지정합니다.
* refit: defalut가 True 이며, True로 생성할 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator객체를 해당 하이퍼파라미터로 재학습시킵니다.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    test_size = 0.2, random_state =121)
dtree = DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth': [1,2,3], 'min_samples_split': [2,3]}

In [3]:
import pandas as pd

#param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정.
#refit = True가 default임.True이면 가장 좋은 파라미털 설정으로 재학습시킴.

grid_dtree = GridSearchCV(dtree, param_grid= parameters, cv= 3, refit= True)

In [4]:
# 붓꽃 학습 데이터로 param_grid의 파라미터를 순차적으로 학습/평가.
grid_dtree.fit(X_train, y_train)

#GridSearchCV 결과를 추출해 DataFrame으로 변환
score_df = pd.DataFrame(grid_dtree.cv_results_)
score_df[['params', 'mean_test_score', 'rank_test_score',
          'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


* parmas 칼럼에는 수행할 때마다 적용된 개별 하이퍼 파라미터 값을 나타냅니다.
* rank_test_score는 하이퍼 파라미터별로 성능이 좋은 score 순위를 나타냅니다. 1위가 가장 뛰어난 순이이며 이때의 파라미터가 최적의 하이퍼 파라미터입니다.
* mean_test_score느 개별 하이퍼 파라미터 별로 cv폴딩 테스트 세트에 대해 총 수행한 평가 평균값입니다.

In [5]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [6]:
# GridSearchCV의 refit으로 이미 학습된 estimatory 변환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 되어있으므로 별도 학습이 필요 없음
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도:{0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도:0.9667
